In [31]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from numpy import random
from scipy.stats import norm
from sklearn.naive_bayes import BernoulliNB

In [32]:
#generate sample data for discrete NB
X = random.choice([0,1], size=(1000, 3))

In [33]:
X

array([[1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       ...,
       [1, 1, 0],
       [1, 0, 1],
       [0, 0, 1]])

In [63]:
class NaiveBayes:
  def __init__(self, X, y, gaussian, laplace) -> None:
    self.X = X
    self.y = y
    self.gaussian = gaussian
    self.laplace = laplace
  
  def splitData(self):
    X_train, X_test, y_train, y_test= train_test_split(self.X, self.y, test_size = 0.3, random_state = 1)
    return X_train, X_test, y_train, y_test

  def fitDistribution(self, data):
    mean = np.mean(data)
    std = np.std(data)
    dist = norm(mean,std)
    return dist

  def probability_gaussian(self, X, dist1, dist2, prior):
    return prior * dist1.pdf(X[0]) * dist2.pdf(X[1])

  def probability_b(self,y, x):
    if y==0:
      #print('{}  {}  {} {}'.format(((X0_10_prob if x[0]==1 else X0_00_prob)),((X1_10_prob if x[1]==1 else X1_00_prob)),(prior_0)))
      return (self.X0_10_prob if x[0]==1 else self.X0_00_prob)*(self.X1_10_prob if x[1]==1 else self.X1_00_prob)*self.prior_0
    elif y==1:
      return (self.X0_11_prob if x[0]==1 else self.X0_01_prob)*(self.X1_11_prob if x[1]==1 else self.X1_01_prob)*self.prior_1

  def fit(self):
    self.X_train, self.X_test, self.y_train, self.y_test = self.splitData()
    
    self.X0_train = self.X_train[self.y_train == 0] 
    self.X1_train = self.X_train[self.y_train == 1]
    
    self.prior_1 = len(self.X1_train)/ len(self.X_train)
    self.prior_0 = len(self.X0_train) / len(self.X_train)
    if self.gaussian == True:
    
      self.X00_dist = self.fitDistribution(self.X0_train[:,0])
      self.X10_dist = self.fitDistribution(self.X0_train[:,1])
      
      self.X01_dist = self.fitDistribution(self.X1_train[:,0])
      self.X11_dist = self.fitDistribution(self.X1_train[:,1])
    
    if self.gaussian == False:
      if self.laplace == True:
        self.X0_10_prob = (len(self.X0_train[np.where(self.X0_train[:,0] == 1)])+1)/(len(self.X0_train)+2)
        self.X0_00_prob = (len(self.X0_train[np.where(self.X0_train[:,0] == 0)])+1)/(len(self.X0_train)+2)

        self.X1_10_prob = (len(self.X0_train[np.where(self.X0_train[:,1] == 1)])+1)/(len(self.X0_train)+2)
        self.X1_00_prob = (len(self.X0_train[np.where(self.X0_train[:,1] == 0)])+1)/(len(self.X0_train)+2)

        self.X0_11_prob = (len(self.X1_train[np.where(self.X1_train[:,0] == 1)])+1)/(len(self.X1_train)+2)
        self.X0_01_prob = (len(self.X1_train[np.where(self.X1_train[:,0] == 0)])+1)/(len(self.X1_train)+2)

        self.X1_11_prob = (len(self.X1_train[np.where(self.X1_train[:,1] == 1)])+1)/(len(self.X1_train)+2)
        self.X1_01_prob = (len(self.X1_train[np.where(self.X1_train[:,1] == 0)])+1)/(len(self.X1_train)+2)

      if self.laplace == False:
        self.X0_10_prob = (len(self.X0_train[np.where(self.X0_train[:,0] == 1)]))/(len(self.X0_train))
        self.X0_00_prob = (len(self.X0_train[np.where(self.X0_train[:,0] == 0)]))/(len(self.X0_train))

        self.X1_10_prob = (len(self.X0_train[np.where(self.X0_train[:,1] == 1)]))/(len(self.X0_train))
        self.X1_00_prob = (len(self.X0_train[np.where(self.X0_train[:,1] == 0)]))/(len(self.X0_train))

        self.X0_11_prob = (len(self.X1_train[np.where(self.X1_train[:,0] == 1)]))/(len(self.X1_train))
        self.X0_01_prob = (len(self.X1_train[np.where(self.X1_train[:,0] == 0)]))/(len(self.X1_train))

        self.X1_11_prob = (len(self.X1_train[np.where(self.X1_train[:,1] == 1)]))/(len(self.X1_train))
        self.X1_01_prob = (len(self.X1_train[np.where(self.X1_train[:,1] == 0)]))/(len(self.X1_train))

  def predict(self):
    right =0
    wrong =0
    for sample, target in zip(self.X_test, self.y_test):
        
        
      if self.gaussian == True:
        py0 = self.probability_gaussian(sample, self.X00_dist, self.X10_dist, self.prior_0)
        py1 = self.probability_gaussian(sample, self.X01_dist, self.X11_dist, self.prior_1)
        print("P(y=0 %s) = %.3f" % (sample, py0 * 100))
        print("P(y=1 %s) = %.3f" % (sample, py1 * 100))
        print("Model predicted class {} and the truth was {}".format(np.argmax ([py0,py1]), target))
        
      if self.gaussian ==False:
        py0 = self.probability_b(0, sample)
        py1 = self.probability_b(1, sample)
        print("P(y=0 %s) = %.3f" % (sample, py0 * 100))
        print("P(y=1 %s) = %.3f" % (sample, py1 * 100))
        print("Model predicted class {} and the truth was {}".format(np.argmax ([py0,py1]), target))

      if np.argmax([py0,py1])==target:
        right+=1
      else: wrong+=1
    print("Right {} , wrong {}".format(right,wrong)) 
    prediction_percent = ((right * 100)/(right+wrong))
    print("% Correctly predicted {} %".format(prediction_percent))

In [64]:
#for Discrete NB Model
clf = NaiveBayes(X = X[:,0:2], y = X[:,2], gaussian = False, laplace =  False)
clf.fit()
clf.predict()

P(y=0 [0 0]) = 13.218

P(y=1 [0 0]) = 11.433

Model predicted class 0 and the truth was 0

P(y=0 [0 1]) = 12.925

P(y=1 [0 1]) = 12.995

Model predicted class 1 and the truth was 1

P(y=0 [0 0]) = 13.218

P(y=1 [0 0]) = 11.433

Model predicted class 0 and the truth was 1

P(y=0 [0 0]) = 13.218

P(y=1 [0 0]) = 11.433

Model predicted class 0 and the truth was 0

P(y=0 [0 1]) = 12.925

P(y=1 [0 1]) = 12.995

Model predicted class 1 and the truth was 1

P(y=0 [1 0]) = 12.496

P(y=1 [1 0]) = 11.567

Model predicted class 0 and the truth was 0

P(y=0 [1 0]) = 12.496

P(y=1 [1 0]) = 11.567

Model predicted class 0 and the truth was 0

P(y=0 [1 0]) = 12.496

P(y=1 [1 0]) = 11.567

Model predicted class 0 and the truth was 0

P(y=0 [1 1]) = 12.218

P(y=1 [1 1]) = 13.147

Model predicted class 1 and the truth was 0

P(y=0 [1 0]) = 12.496

P(y=1 [1 0]) = 11.567

Model predicted class 0 and the truth was 1

P(y=0 [0 0]) = 13.218

P(y=1 [0 0]) = 11.433

Model predicted class 0 and the truth was 1


In [65]:
#for Discrete NB Model with laplace
clf = NaiveBayes(X = X[:,0:2],y = X[:,2], gaussian = False, laplace = True)
clf.fit()
clf.predict()

P(y=0 [0 0]) = 13.215

P(y=1 [0 0]) = 11.438

Model predicted class 0 and the truth was 0

P(y=0 [0 1]) = 12.923

P(y=1 [0 1]) = 12.991

Model predicted class 1 and the truth was 1

P(y=0 [0 0]) = 13.215

P(y=1 [0 0]) = 11.438

Model predicted class 0 and the truth was 1

P(y=0 [0 0]) = 13.215

P(y=1 [0 0]) = 11.438

Model predicted class 0 and the truth was 0

P(y=0 [0 1]) = 12.923

P(y=1 [0 1]) = 12.991

Model predicted class 1 and the truth was 1

P(y=0 [1 0]) = 12.497

P(y=1 [1 0]) = 11.571

Model predicted class 0 and the truth was 0

P(y=0 [1 0]) = 12.497

P(y=1 [1 0]) = 11.571

Model predicted class 0 and the truth was 0

P(y=0 [1 0]) = 12.497

P(y=1 [1 0]) = 11.571

Model predicted class 0 and the truth was 0

P(y=0 [1 1]) = 12.221

P(y=1 [1 1]) = 13.142

Model predicted class 1 and the truth was 0

P(y=0 [1 0]) = 12.497

P(y=1 [1 0]) = 11.571

Model predicted class 0 and the truth was 1

P(y=0 [0 0]) = 13.215

P(y=1 [0 0]) = 11.438

Model predicted class 0 and the truth was 1
